# Notebook 4: Autonomous Driving Dataset Exploration

**Session 1: AI-based Perception Systems in Autonomous Vehicles**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_1_AI_Perception_Systems/notebooks/04_Dataset_Exploration.ipynb)

**Author:** Milin Patel  
**Duration:** ~20 minutes

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand major autonomous driving datasets (KITTI, nuScenes, Waymo)
- ✅ Explore dataset structure and annotations
- ✅ Visualize sample driving scenes with labels
- ✅ Compare dataset characteristics and use cases
- ✅ Analyze class distributions and data diversity
- ✅ Understand the importance of quality datasets for AV safety

---

## 📦 Setup and Imports

This cell checks if you're running on Google Colab and installs necessary dependencies.

In [ ]:
import sys
import os

# Check if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running on Google Colab - Installing dependencies...\n")
    !pip install -q opencv-python matplotlib numpy pillow requests pandas seaborn
    print("✅ Setup complete!\n")
else:
    print("💻 Running locally\n")

print("✅ Environment ready!")

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
from PIL import Image
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

---

## 1️⃣ Overview of Major AV Datasets

Training robust perception systems requires **large-scale, diverse, and accurately labeled datasets**.

### Major Public Datasets:

| Dataset | Year | Scenes | Frames | Sensors | 3D Annotations | Key Features |
|---------|------|--------|--------|---------|----------------|---------------|
| **KITTI** | 2012 | 200+ | 200K+ | Camera, LiDAR, GPS | ✅ | First large-scale 3D dataset |
| **nuScenes** | 2019 | 1,000 | 1.4M | 6 cameras, 5 radars, LiDAR | ✅ | Full 360° coverage, diverse cities |
| **Waymo Open** | 2019 | 2,000+ | 200K+ | 5 cameras, 5 LiDARs | ✅ | Largest, highest quality |
| **BDD100K** | 2020 | 100K | 100M | Camera | ❌ (2D only) | Diverse weather, time, location |
| **Argoverse** | 2019 | 300+ | 300K+ | Cameras, LiDAR | ✅ | HD maps, trajectory prediction |

**Today's focus:** KITTI and nuScenes (most widely used in research)

In [ ]:
# Create comprehensive comparison table
dataset_comparison = pd.DataFrame({
    'Dataset': ['KITTI', 'nuScenes', 'Waymo Open', 'BDD100K', 'Argoverse'],
    'Release Year': [2012, 2019, 2019, 2020, 2019],
    'Total Frames': ['200K+', '1.4M', '200K+', '100M', '300K+'],
    'Cameras': [4, 6, 5, 1, 7],
    'LiDAR': ['1 (64-beam)', '1 (32-beam)', '5 (64-beam)', 'None', '2'],
    'Radar': ['None', '5', 'None', 'None', 'None'],
    '3D Boxes': [True, True, True, False, True],
    'Weather Diversity': ['Low', 'High', 'Medium', 'High', 'Medium'],
    'Geographic Coverage': ['Germany', 'USA, Singapore', 'USA', 'USA', 'USA'],
    'Size (GB)': [150, 500, 1000, 100, 300]
})

display(dataset_comparison)

print("\n💡 Key Observations:")
print("   - Waymo Open: Largest and highest quality (but 1TB!)")
print("   - nuScenes: Best sensor diversity (cameras + radar + LiDAR)")
print("   - KITTI: Oldest but still widely used for benchmarking")
print("   - BDD100K: Largest image count, but no 3D data")

### Visualize Dataset Statistics

In [ ]:
# Visualize key statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Total frames comparison
datasets = ['KITTI', 'nuScenes', 'Waymo', 'BDD100K', 'Argoverse']
frames = [200000, 1400000, 200000, 100000000, 300000]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']

axes[0, 0].barh(datasets, frames, color=colors)
axes[0, 0].set_xlabel('Number of Frames', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Dataset Size Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xscale('log')
for i, v in enumerate(frames):
    axes[0, 0].text(v * 1.1, i, f'{v/1e6:.1f}M' if v >= 1e6 else f'{v/1e3:.0f}K', 
                    va='center', fontweight='bold')

# 2. Sensor coverage
sensor_data = {
    'KITTI': {'Camera': 4, 'LiDAR': 1, 'Radar': 0},
    'nuScenes': {'Camera': 6, 'LiDAR': 1, 'Radar': 5},
    'Waymo': {'Camera': 5, 'LiDAR': 5, 'Radar': 0},
    'BDD100K': {'Camera': 1, 'LiDAR': 0, 'Radar': 0},
    'Argoverse': {'Camera': 7, 'LiDAR': 2, 'Radar': 0}
}

x = np.arange(len(datasets))
width = 0.25
cameras = [sensor_data[d]['Camera'] for d in datasets]
lidars = [sensor_data[d]['LiDAR'] for d in datasets]
radars = [sensor_data[d]['Radar'] for d in datasets]

axes[0, 1].bar(x - width, cameras, width, label='Cameras', color='#FF6B6B')
axes[0, 1].bar(x, lidars, width, label='LiDARs', color='#4ECDC4')
axes[0, 1].bar(x + width, radars, width, label='Radars', color='#45B7D1')
axes[0, 1].set_xlabel('Dataset', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Number of Sensors', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Sensor Coverage by Dataset', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(datasets, rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Geographic coverage
locations = ['Germany', 'USA', 'Singapore']
coverage = {
    'KITTI': [1, 0, 0],
    'nuScenes': [0, 1, 1],
    'Waymo': [0, 1, 0],
    'BDD100K': [0, 1, 0],
    'Argoverse': [0, 1, 0]
}

coverage_matrix = np.array([coverage[d] for d in datasets])
im = axes[1, 0].imshow(coverage_matrix, cmap='YlGn', aspect='auto')
axes[1, 0].set_xticks(range(len(locations)))
axes[1, 0].set_xticklabels(locations)
axes[1, 0].set_yticks(range(len(datasets)))
axes[1, 0].set_yticklabels(datasets)
axes[1, 0].set_title('Geographic Coverage', fontsize=14, fontweight='bold')

# Add text annotations
for i in range(len(datasets)):
    for j in range(len(locations)):
        text = axes[1, 0].text(j, i, '✓' if coverage_matrix[i, j] else '',
                              ha="center", va="center", color="darkgreen", 
                              fontsize=20, fontweight='bold')

# 4. Dataset size (storage)
sizes_gb = [150, 500, 1000, 100, 300]
axes[1, 1].pie(sizes_gb, labels=datasets, autopct='%1.1f%%', 
               colors=colors, startangle=90)
axes[1, 1].set_title('Storage Requirements (Total: 2.05TB)', 
                     fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Dataset characteristics vary significantly!")
print("   Choice depends on: research goal, computational resources, geographic coverage needed")

---

## 2️⃣ KITTI Dataset Deep Dive

**KITTI Vision Benchmark Suite** (2012) - Karlsruhe Institute of Technology

### Key Features:
- **Location:** Karlsruhe, Germany (mid-size city)
- **Duration:** Hours of driving
- **Sensors:**
  - 2 color cameras (stereo)
  - 2 grayscale cameras (stereo)
  - Velodyne HDL-64E LiDAR (64 beams)
  - GPS/IMU
- **Tasks:** 3D object detection, tracking, depth estimation, odometry, segmentation

### Class Distribution (3D Object Detection):

In [ ]:
# Approximate KITTI training set class distribution
kitti_classes = {
    'Car': 28742,
    'Pedestrian': 4487,
    'Cyclist': 1627,
    'Van': 2914,
    'Truck': 1094,
    'Tram': 511,
    'Person_sitting': 222,
    'Misc': 973
}

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
classes = list(kitti_classes.keys())
counts = list(kitti_classes.values())
colors_bar = plt.cm.Set3(range(len(classes)))

bars = axes[0].bar(classes, counts, color=colors_bar)
axes[0].set_xlabel('Object Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Instances', fontsize=12, fontweight='bold')
axes[0].set_title('KITTI Class Distribution (Training Set)', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Add count labels
for bar in bars:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height):,}', ha='center', va='bottom', fontsize=9)

# Pie chart for proportions
axes[1].pie(counts, labels=classes, autopct='%1.1f%%', 
            colors=colors_bar, startangle=90)
axes[1].set_title('Class Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ Class Imbalance Observation:")
print(f"   - Most common: Car ({kitti_classes['Car']:,} instances)")
print(f"   - Least common: Person_sitting ({kitti_classes['Person_sitting']} instances)")
print(f"   - Imbalance ratio: {kitti_classes['Car']/kitti_classes['Person_sitting']:.1f}:1")
print("\n💡 This imbalance affects model performance!")
print("   Models tend to perform worse on rare classes (e.g., Person_sitting)")

### Visualize Sample KITTI Scene

Since we can't download the full KITTI dataset here, let's create a representative visualization:

In [ ]:
# Create synthetic KITTI-style visualization
def create_kitti_style_scene():
    """Create a synthetic KITTI-style driving scene with annotations"""
    # Create base image
    img_width, img_height = 1242, 375
    image = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    
    # Sky
    image[:img_height//3] = [135, 206, 235]
    
    # Road
    image[img_height//3:] = [105, 105, 105]
    
    # Add lane markings
    for i in range(0, img_width, 80):
        cv2.rectangle(image, (i, img_height-50), (i+40, img_height-45), (255, 255, 255), -1)
    
    # Add buildings
    cv2.rectangle(image, (50, 50), (200, img_height//3), (169, 169, 169), -1)
    cv2.rectangle(image, (900, 70), (1100, img_height//3), (139, 139, 139), -1)
    
    # Simulate objects with bounding boxes
    objects = [
        {'class': 'Car', 'bbox': [400, 180, 200, 120], 'color': (0, 255, 0)},
        {'class': 'Car', 'bbox': [700, 200, 180, 100], 'color': (0, 255, 0)},
        {'class': 'Pedestrian', 'bbox': [250, 200, 50, 120], 'color': (255, 0, 0)},
        {'class': 'Cyclist', 'bbox': [550, 210, 60, 100], 'color': (255, 165, 0)},
    ]
    
    # Draw objects
    for obj in objects:
        x, y, w, h = obj['bbox']
        # Draw filled rectangle for object
        cv2.rectangle(image, (x, y), (x+w, y+h), (80, 80, 80), -1)
        # Draw bounding box
        cv2.rectangle(image, (x, y), (x+w, y+h), obj['color'], 3)
        # Add label
        cv2.putText(image, obj['class'], (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, obj['color'], 2)
    
    return image, objects

# Generate and visualize
kitti_image, annotations = create_kitti_style_scene()

plt.figure(figsize=(16, 6))
plt.imshow(kitti_image)
plt.title('KITTI-Style Scene with 3D Object Annotations', fontsize=14, fontweight='bold')
plt.axis('off')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='green', label='Car'),
    Patch(facecolor='red', label='Pedestrian'),
    Patch(facecolor='orange', label='Cyclist')
]
plt.legend(handles=legend_elements, loc='upper right', fontsize=12)
plt.tight_layout()
plt.show()

print("\n📊 Detected Objects:")
for i, obj in enumerate(annotations, 1):
    print(f"   {i}. {obj['class']}: Bbox {obj['bbox']}")

---

## 3️⃣ nuScenes Dataset Deep Dive

**nuScenes** (2019) - Motional (formerly nuTonomy)

### Key Improvements over KITTI:
- ✅ **Full 360° coverage** (6 cameras)
- ✅ **Radar data** (5 radars for all-weather perception)
- ✅ **Diverse locations** (Boston, Singapore)
- ✅ **More weather conditions** (rain, night)
- ✅ **Longer sequences** (20 seconds each)
- ✅ **23 object classes** (vs. 8 in KITTI)

### Sensor Setup:

In [ ]:
# nuScenes sensor configuration
print("🚗 nuScenes Sensor Suite:\n")
print("📷 Cameras (6):")
cameras = [
    'CAM_FRONT',
    'CAM_FRONT_LEFT',
    'CAM_FRONT_RIGHT',
    'CAM_BACK',
    'CAM_BACK_LEFT',
    'CAM_BACK_RIGHT'
]
for cam in cameras:
    print(f"   - {cam}")

print("\n📡 Radars (5):")
radars = [
    'RADAR_FRONT',
    'RADAR_FRONT_LEFT',
    'RADAR_FRONT_RIGHT',
    'RADAR_BACK_LEFT',
    'RADAR_BACK_RIGHT'
]
for radar in radars:
    print(f"   - {radar}")

print("\n🌐 LiDAR (1):")
print("   - LIDAR_TOP (32-beam, 360°)")

# Visualize sensor placement
fig, ax = plt.subplots(figsize=(10, 10))

# Draw vehicle (top-down view)
vehicle = plt.Rectangle((-1, -2), 2, 4, fill=True, color='lightgray', edgecolor='black', linewidth=2)
ax.add_patch(vehicle)

# Camera positions and FOV
camera_positions = {
    'FRONT': (0, 2.5, 70),
    'FRONT_LEFT': (-1.5, 1.5, 70),
    'FRONT_RIGHT': (1.5, 1.5, 70),
    'BACK': (0, -2.5, 70),
    'BACK_LEFT': (-1.5, -1.5, 70),
    'BACK_RIGHT': (1.5, -1.5, 70)
}

for name, (x, y, fov) in camera_positions.items():
    ax.plot(x, y, 'ro', markersize=10)
    ax.text(x, y+0.5, name.replace('_', '\n'), ha='center', fontsize=8, fontweight='bold')

# LiDAR at center
ax.plot(0, 0, 'g^', markersize=20, label='LiDAR (360°)')

ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('Lateral Distance (m)', fontsize=12)
ax.set_ylabel('Longitudinal Distance (m)', fontsize=12)
ax.set_title('nuScenes Sensor Placement (Top-Down View)', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Full 360° coverage eliminates blind spots!")

### nuScenes Class Distribution

In [ ]:
# nuScenes class categories (23 classes grouped)
nuscenes_classes = {
    'Vehicles': {
        'car': 385000,
        'truck': 55000,
        'bus': 18000,
        'trailer': 23000,
        'construction_vehicle': 12000,
        'motorcycle': 8500,
        'bicycle': 9200
    },
    'Humans': {
        'pedestrian': 96000
    },
    'Objects': {
        'traffic_cone': 45000,
        'barrier': 50000
    }
}

# Flatten for visualization
all_classes = {}
for category, classes in nuscenes_classes.items():
    all_classes.update(classes)

# Sort by count
sorted_classes = dict(sorted(all_classes.items(), key=lambda x: x[1], reverse=True))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
classes_list = list(sorted_classes.keys())
counts_list = list(sorted_classes.values())

axes[0].barh(classes_list, counts_list, color=plt.cm.viridis(np.linspace(0, 1, len(classes_list))))
axes[0].set_xlabel('Number of Instances', fontsize=12, fontweight='bold')
axes[0].set_title('nuScenes Class Distribution', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Add counts
for i, (cls, count) in enumerate(sorted_classes.items()):
    axes[0].text(count + 5000, i, f'{count/1000:.0f}K', va='center', fontsize=9)

# Category breakdown
category_totals = {cat: sum(classes.values()) for cat, classes in nuscenes_classes.items()}
axes[1].pie(category_totals.values(), labels=category_totals.keys(), 
            autopct='%1.1f%%', startangle=90, colors=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[1].set_title('Class Categories', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 nuScenes has more balanced class distribution than KITTI")
print("   But still biased toward common objects (cars, pedestrians)")

---

## 4️⃣ Dataset Quality and Diversity Analysis

**Why dataset quality matters for safety:**
- Models only learn what they see in training data
- Rare scenarios (edge cases) often missing
- Geographic/weather bias affects generalization

In [ ]:
# Compare dataset diversity dimensions
diversity_metrics = pd.DataFrame({
    'Dimension': ['Weather Conditions', 'Time of Day', 'Geographic Diversity', 
                  'Traffic Density', 'Road Types', 'Object Classes'],
    'KITTI': ['Low (mostly clear)', 'Day only', 'Single city', 'Medium', 'Urban/Highway', '8 classes'],
    'nuScenes': ['High (rain, night)', 'Day & Night', '2 cities, 2 countries', 'High', 'Urban/Highway', '23 classes'],
    'Waymo Open': ['Medium', 'Day & Night', 'Multiple US cities', 'High', 'Diverse', '4 main classes'],
    'BDD100K': ['High', 'Day & Night', 'USA-wide', 'Very High', 'Very Diverse', '10 classes']
})

display(diversity_metrics)

print("\n⚠️ Diversity Gaps Lead to Safety Issues:")
print("   1. Tesla Autopilot struggles in heavy rain (trained mostly on clear weather)")
print("   2. Systems fail in new cities with different road layouts")
print("   3. Rare objects (animals, debris) often missed")
print("\n💡 Solution: Collect diverse data + data augmentation + synthetic data")

### Visualize Diversity Scores

In [ ]:
# Assign diversity scores (0-10 scale)
datasets_diversity = ['KITTI', 'nuScenes', 'Waymo', 'BDD100K']
diversity_scores = {
    'Weather': [3, 8, 6, 9],
    'Time': [3, 8, 7, 8],
    'Geography': [2, 6, 7, 10],
    'Traffic': [6, 8, 8, 9],
    'Road Types': [6, 7, 8, 10],
    'Objects': [5, 9, 6, 7]
}

# Radar chart
angles = np.linspace(0, 2 * np.pi, len(diversity_scores), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for i, dataset in enumerate(datasets_diversity):
    values = [diversity_scores[dim][i] for dim in diversity_scores.keys()]
    values += values[:1]  # Complete the circle
    ax.plot(angles, values, 'o-', linewidth=2, label=dataset)
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(diversity_scores.keys(), fontsize=11)
ax.set_ylim(0, 10)
ax.set_yticks([2, 4, 6, 8, 10])
ax.set_yticklabels(['2', '4', '6', '8', '10'], fontsize=9)
ax.set_title('Dataset Diversity Comparison (Radar Chart)', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=12)
ax.grid(True)

plt.tight_layout()
plt.show()

print("\n💡 Ideal dataset: Large area covering all dimensions (approaching 10 on all axes)")
print("   No single dataset is perfect - consider combining multiple sources!")

---

## 5️⃣ Edge Cases and Long-Tail Distribution

**The Long-Tail Problem:**
- Most scenarios are common (highway driving, clear weather)
- Rare scenarios (animals, debris, unusual weather) are underrepresented
- But rare scenarios are often safety-critical!

In [ ]:
# Simulate long-tail distribution of driving scenarios
scenarios = [
    'Highway cruising',
    'Urban driving',
    'Intersections',
    'Parking lots',
    'Night driving',
    'Light rain',
    'Heavy rain',
    'Construction zones',
    'Emergency vehicles',
    'Animals crossing',
    'Debris on road',
    'Snow/ice',
    'Fog',
    'Unusual objects'
]

# Frequency (log scale)
frequencies = [100000, 80000, 60000, 40000, 20000, 10000, 3000, 2000, 1000, 500, 300, 200, 150, 50]

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Linear scale
colors_scenarios = ['green' if f > 10000 else 'orange' if f > 1000 else 'red' for f in frequencies]
axes[0].bar(range(len(scenarios)), frequencies, color=colors_scenarios)
axes[0].set_xticks(range(len(scenarios)))
axes[0].set_xticklabels(scenarios, rotation=45, ha='right')
axes[0].set_ylabel('Frequency in Dataset', fontsize=12, fontweight='bold')
axes[0].set_title('Scenario Distribution in Training Data (Linear Scale)', 
                  fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Log scale
axes[1].bar(range(len(scenarios)), frequencies, color=colors_scenarios)
axes[1].set_yscale('log')
axes[1].set_xticks(range(len(scenarios)))
axes[1].set_xticklabels(scenarios, rotation=45, ha='right')
axes[1].set_ylabel('Frequency (log scale)', fontsize=12, fontweight='bold')
axes[1].set_title('Scenario Distribution (Log Scale) - Reveals Long Tail', 
                  fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='green', label='Common (>10K)'),
    Patch(facecolor='orange', label='Uncommon (1K-10K)'),
    Patch(facecolor='red', label='Rare (<1K)')
]
axes[1].legend(handles=legend_elements, loc='upper right', fontsize=11)

plt.tight_layout()
plt.show()

print("\n⚠️ The Long-Tail Problem:")
print(f"   - Common scenarios: {sum(frequencies[:4])/sum(frequencies)*100:.1f}% of data")
print(f"   - Rare scenarios: {sum(frequencies[9:])/sum(frequencies)*100:.1f}% of data")
print("\n💡 But rare scenarios cause accidents!")
print("   - Need targeted data collection for edge cases")
print("   - Simulation helps fill gaps (CARLA, LGSVL)")

---

## ✏️ Exercise: Dataset Selection

**Scenario:** You're developing a perception system for an autonomous shuttle that will operate:
- **Location:** University campus (urban, low speed)
- **Speed:** Max 25 km/h
- **Weather:** All seasons (including snow)
- **Time:** Day and night
- **Key risks:** Pedestrians, cyclists, unexpected obstacles

**Question:** Which dataset(s) would you choose and why? What are the gaps?

In [ ]:
# TODO: Fill in your analysis
your_choice = {
    'primary_dataset': '',  # e.g., 'nuScenes'
    'secondary_datasets': [],  # e.g., ['KITTI', 'BDD100K']
    'reasoning': '',  # Explain your choice
    'identified_gaps': [],  # What's missing?
    'mitigation_strategy': ''  # How to address gaps?
}

print("💡 Consider:")
print("   - Sensor coverage needed")
print("   - Speed range (campus is low-speed)")
print("   - Weather diversity")
print("   - Pedestrian/cyclist density")
print("   - Dataset size (computational constraints)")

# Uncomment to see sample answer
# sample_answer = {
#     'primary_dataset': 'nuScenes',
#     'secondary_datasets': ['BDD100K'],
#     'reasoning': 'nuScenes has diverse weather, night data, and good pedestrian coverage. BDD100K adds more diversity.',
#     'identified_gaps': ['Low-speed campus scenarios', 'Dense pedestrian crowds', 'Campus-specific objects (scooters, etc.)'],
#     'mitigation_strategy': 'Collect custom campus data, use simulation (CARLA) for edge cases, data augmentation'
# }
# print("\n Sample Answer:", sample_answer)

---

## 🎯 Key Takeaways

### Dataset Importance
- **Quality > Quantity:** Diverse, accurately labeled data beats large but biased datasets
- **No perfect dataset:** Each has strengths and weaknesses
- **Combine sources:** Use multiple datasets to improve coverage

### Major Datasets Compared
- **KITTI:** Pioneer, still used for benchmarks, limited diversity
- **nuScenes:** Best sensor variety, good weather/time diversity, 360° coverage
- **Waymo Open:** Largest, highest quality, but USA-centric
- **BDD100K:** Most geographic diversity, but camera-only (no 3D)

### Safety Implications
- **Class imbalance:** Models worse on rare but critical objects
- **Long-tail distribution:** Rare scenarios cause accidents
- **Geographic bias:** Models fail in new locations
- **Weather gaps:** Missing conditions lead to failures

### Best Practices
1. **Understand ODD:** Choose datasets matching deployment environment
2. **Identify gaps:** What scenarios are missing?
3. **Collect targeted data:** Fill critical gaps
4. **Use simulation:** Generate edge cases (more in Session 2!)
5. **Validate thoroughly:** Test on held-out data from different sources

---

## 🔜 Next: Sensor Fusion Basics

Now that we understand datasets, let's learn how to fuse data from multiple sensors!

**Open Notebook 5:** `05_Sensor_Fusion_Basics.ipynb`

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-17*